In [ ]:
"""
You are given an n x n integer matrix grid where each value grid[i][j] 
represents the elevation at that point (i, j).

The rain starts to fall. At time t, the depth of the water everywhere 
is t. You can swim from a square to another 4-directionally adjacent 
square if and only if the elevation of both squares individually are 
at most t. You can swim infinite distances in zero time. Of course, 
you must stay within the boundaries of the grid during your swim.

Return the least time until you can reach the bottom right square 
(n - 1, n - 1) if you start at the top left square (0, 0).

 

Example 1:


    Input: 
        grid = [[0,2],[1,3]]
    Output: 
        3
    Explanation:
        At time 0, you are in grid location (0, 0).
        You cannot go anywhere else because 4-directionally adjacent 
        neighbors have a higher elevation than t = 0.

        You cannot reach point (1, 1) until time 3.
        When the depth of water is 3, we can swim anywhere inside the grid.


Example 2:
    Input: grid = [[0,1,2,3,4],[24,23,22,21,5],[12,13,14,15,16],[11,17,18,19,20],[10,9,8,7,6]]
    Output: 16
    Explanation: 
        The final route is shown.
        We need to wait until time 16 so that (0, 0) and (4, 4) are connected.
 

Constraints:

n == grid.length
n == grid[i].length
1 <= n <= 50
0 <= grid[i][j] < n2
Each value grid[i][j] is unique.
"""
"""
n * n
{0 -> n*n-1}

TIP::

PS: we need to find at what time we will be able to reach 
the last location starting from the first location such that 
time taken is minimum.
obviously we can't reach any lesser than the height of the last 
location and not any longer than the max height.
{Intuition for binary search on answer approach}
Intutively thinking about the union and find approach is hard.

Would describe the approaches:

Binary Search:-
===============
Bascially if we could start from the very first and keep on 
increasing time and see if we can reach the last , if at any 
time we would be able to reach the last then that time would be minimum.

DFS:-
====
Improved version of the prev approach {DFS+binary search}
we know that our ans would always be in the range{last height,max heigh}
we can apply binary search for the answer in this range, and pick 
the approapriate answer, rather than trying for all the possible times.

Dijkstra:-
==========
we keep on selecting the min, and have a priority queue for 
the min {rather than selecting all in dfs approach},
in dikstra we get a path which contains all the min possible 
nodes to reach the end.
since the max of the nodes would be the least time required to visit 
this particular path given by the dikstra algorithm so we could 
be having the max of it as our time.

Union and find intution:-
=========================
Now here we can see the connectivity, Union and find intution, 
all the elements got connected at last, see if we could be 
connecting all the elements in increasing time, and at a particular 
time we see that the first and the last element also got connected 
then since we are increasing the time linearly
we must have got the min time.

Approach
=========

Union and find approach
-----------------------

1.start from time 0,
2. see the elements that can swim at this time.
3. mark it as visited
4. find it's location {need a method to store the location of elements which can be reached exactly at time t}
5. start from the first location,
6. mark this location as visited
7. See its neighbours , if anoyone already visited that means they are in the path and should be connected to it. Connect both the components
8. if anytime you find that the first and last location got connected
this particular time should be the min as we came here first time only.
 
Complexity
==========
Time complexity: n^2*log(n)
    nsquare : for time traversal
    logn: max time to update the parents in find operation.

Space complexity:
=================
O(n square): visited array,disjoint array
we can instead of visited array either use the grid array itself
or since any element value less than time t will be already  visited.


"""

# Best => Union-Find => With per unit increase in time, keep 
# unifying nodes. Once we are able to unify last node then that shold be minimum time.
class Solution:
    def swimInWater(self, grid) -> int:
        N = len(grid)
        first, last = 0, N*N-1

        np = list(range(last+1))
        rp = [0]*(last+1)
        
        def findp(node):
            px = np[node]
            if px == node:
                return px
            np[node] = findp(px)
            return np[node]

        def connect(n1, n2):
            px, py = findp(n1), findp(n2)
            if px == py:
                return
            rx, ry = rp[px], rp[py]
            if rx < ry:
                np[px] = py
            elif ry < rx:
                np[py] = px
            else:
                np[px] = py
                rp[py] += 1

        reachable = {grid[i][j]: (i, j) for i in range(N) for j in range(N)}
        seen = [[0 for i in range(N)] for j in range(N)]

        for k in range(last+1):
            x, y = reachable[k]
            node = x*N + y
            seen[x][y] = 1
            for i, j in [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
                if 0<=i<N and 0<=j<N and seen[i][j]==1:
                    connect(node, i*N+j)
            if findp(first) == findp(last):
                return k


# Relatviely better - dijkastra variation
from heapq import heapify, heappush, heappop
class Solution:
    def swimInWater(self, grid) -> int:
        R = len(grid)
        C = len(grid[0])
        seen = set()
        dist = [[float('inf') for y in range(C)] for x in range(R)]
        dist[0][0] = grid[0][0]
        pq = [(dist[0][0], (0, 0))]
        while pq:
            xy_wt, (x, y) = heappop(pq)
            seen.add((x, y))
            if dist[x][y] != float('inf') and xy_wt > dist[x][y]:
                continue
            for i, j in ((x+1, y), (x-1, y), (x, y+1), (x, y-1)):
                if i < 0 or i >= R or j < 0 or j >= C or (i, j) in seen:
                    continue
                new_d = max(xy_wt, grid[i][j])
                if new_d < dist[i][j]:
                    dist[i][j] = new_d
                    heappush(pq, (dist[i][j], (i, j)))
        return dist[R-1][C-1]

# Too slow !!
from functools import cache
class Solution:
    def swimInWater(self, grid) -> int:
        rmax = []
        R = len(grid)
        C = len(grid[0])
        seen = set()
        @cache
        def find_path(x, y, curr_mx):
            nonlocal rmax
            if x == R-1 and y == C-1:
                rmax.append(curr_mx)
                return
            if (
                x < 0 or x >= R or y < 0 or y >= C or
                (x, y) in seen):
                return
            seen.add((x, y))
            temp = max(curr_mx, grid[x][y])
            find_path(x+1, y, temp)
            find_path(x-1, y, temp)
            find_path(x, y+1, temp)
            find_path(x, y-1, temp)
            seen.remove((x, y))
        find_path(0, 0, 0)
        return max(min(rmax), grid[R-1][C-1])
        